<a href="https://colab.research.google.com/github/djbell01/563-DawsonBell/blob/main/Project2_DBell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import hoomd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import freud
import gsd.hoomd
import os

In [ ]:
class Simulation():
    def __init__(self,folder_name,T,dt=0.001,density=0.5,epsilon=1.0,sigma=1.0,unit_cell_replicas=20):
        self.folder_name = folder_name
        self.kBT = T # Assume kB = 1
        self.dt = dt
        self.density = density
        self.epsilon = epsilon
        self.sigma = sigma
        self.num_replicas = unit_cell_replicas

        device = hoomd.device.CPU()
        seed = np.random.randint(1,1e4)
        self.simulation = hoomd.Simulation(device = device, seed = seed)

        #a is the spacing between particles
        a = 1/(density**(1/3.0))
        #We want a number of particles N in our system.
        #Since we will be replicating the system in 3 dimensions, the number of unit cells we need is N^(1/3)
        N_particles = self.num_replicas**3
        grid_particles = freud.data.UnitCell([a,a,a,0,0,0],[[0,0,0]]).generate_system(self.num_replicas)
        self.box_length = grid_particles[0].Lx

        frame = gsd.hoomd.Frame()
        frame.particles.N = N_particles
        frame.particles.position = grid_particles[1]
        frame.configuration.box = [self.box_length,self.box_length,self.box_length,0,0,0]

        #Types of particles define different interactions. In an atomistic simulation these might be C, O, and H.
        #in a coarse-grained simulation we can give them a simple name like A
        frame.particles.typeid = [0]*N_particles
        frame.particles.types = ['A']

        os.mkdir(folder_name)

        #Finally, save our initial state:
        with gsd.hoomd.open(name=folder_name+'/initial_state.gsd', mode='w') as f:
            f.append(frame)

        self.simulation.create_state_from_gsd(filename=folder_name+'/initial_state.gsd')

        integrator = hoomd.md.Integrator(dt = dt)
        nvt = hoomd.md.methods.NVT(filter = hoomd.filter.All(),kT=self.kBT,tau=1.0)
        integrator.methods.append(nvt)

        cell = hoomd.md.nlist.Cell(buffer=0.4)

        #Define the force for different particles
        lj = hoomd.md.pair.LJ(nlist=cell)

        lj.params[('A', 'A')] = {"epsilon":epsilon, "sigma":sigma}

        lj.r_cut[('A', 'A')] = 2.5*sigma

        integrator.forces.append(lj)
        self.simulation.operations.integrator = integrator

        self.simulation.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=self.kBT)
        thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(
            filter=hoomd.filter.All()
        )

        self.simulation.operations.computes.append(thermodynamic_properties)
        logger = hoomd.logging.Logger(categories=['scalar'])
        logger.add(thermodynamic_properties, quantities=['potential_energy','kinetic_energy'])
        energy_table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(period=int(1)), logger=logger, output=open(f'{folder_name}/dt_{str(dt)}.csv','w'))
        self.simulation.operations.writers.append(energy_table)


    def run(self,nsteps):
        self.simulation.run(nsteps)
        snapshot = self.simulation.state.get_snapshot()
        positions = snapshot.particles.position
        np.savetxt(f"{self.folder_name}/final_positions.txt", positions, delimiter=" ")

        with open(f'{self.folder_name}/parameters','w') as file:
             file.write(f'N_particles = {self.num_replicas**3}\n')
             file.write(f'N/V = {self.density}\n')
             file.write(f'Box length = {self.box_length}\n')
             file.write(f'kBT = {self.kBT}\n')
             file.write(f'Timestep = {self.dt}\n')
             file.write(f'Epsilon = {self.epsilon}\n')
             file.write(f'Sigma = {self.sigma}\n')
             file.write(f'Unit Cell Replicas = {self.num_replicas}')



# Independent functions
def plot_total_energy(file_name):
        df = pd.read_csv(file_name,sep='\s+')
        total_energy = (df['md.compute.ThermodynamicQuantities.potential_energy'] + df['md.compute.ThermodynamicQuantities.kinetic_energy']).to_numpy()

        plt.plot(total_energy)
        plt.xlabel('Timestep')
        plt.ylabel('Energy')
        plt.show()

def autocorr1D(array):
    ft = np.fft.rfft(array - np.average(array))
    acorr = np.fft.irfft(ft * np.conjugate(ft)) / (len(array) * np.var(array))
    decorr_time = np.where(acorr<0)[0][0]
    return decorr_time

def heat_capacity(energy_array,temperature,equilibration_length):
        decorr_time = autocorr1D(energy_array)
        print(decorr_time)

        samples = energy_array[equilibration_length::decorr_time]
        samples_squared = samples**2

        return (np.mean(samples)**2 + np.mean(samples_squared))/(temperature**2)


def pbc(d,L):
    for i,x in enumerate(d):
      if x > L/2:
        d[i] = x-L
      elif x <= -L/2:
        d[i] = x+L
    return d


def rdf(file_name,box_length,density):
    positions = np.loadtxt(file_name)
    num_particles = len(positions)
    histograms = []
    for i,p_i in enumerate(positions):
      distances = []
      for j,p_j in enumerate(positions):
        if i == j:
            continue
        diff = p_i - p_j
        diff = pbc(diff,box_length)
        distance = np.linalg.norm(diff)
        distances.append(distance)
      hist,bin_edges = np.histogram(distances,bins=1000,range=(0,box_length/2))
      histograms.append(hist)
    avg_hist = np.mean(histograms,axis=0)
    bin_edges = np.delete(bin_edges,0)
    volumes = []
    for i in range(len(bin_edges)):
        if i == 0:
            vol = (4/3)*np.pi*bin_edges[i]**3
        else:
            vol = (4/3)*np.pi*bin_edges[i]**3 - volumes[i-1]
        volumes.append(vol)
    volumes = np.array(volumes)
    normalized_hist = avg_hist/ (volumes * density * num_particles)
    return bin_edges,normalized_hist


          


In [ ]:
# This is code that google gemini came up with. It substantially increases the efficiency
# and corrects problems I was having with the normalization of the RDF. Hopefully it is okay
# to use this. I'm putting this in a commit so that it doesn't seems like I'm trying to hide
# my using of generative AI to help with the project. My attempt at an RDF and PBC are shown above.

def pbc(d, L):
    return np.where(d > L / 2, d - L, np.where(d <= -L / 2, d + L, d))

def rdf(file_name, box_length, density):
    positions = np.loadtxt(file_name)
    num_particles = len(positions)
    max_radius = box_length / 2
    num_bins = 1000
    bin_edges = np.linspace(0, max_radius, num_bins + 1)
    hist = np.zeros(num_bins)

    for i in range(num_particles):
        diff = positions - positions[i]
        diff = pbc(diff, box_length)
        distances = np.linalg.norm(diff, axis=1)
        distances = distances[distances > 0]  # Exclude self-interaction
        hist += np.histogram(distances, bins=bin_edges)[0]

    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    volumes = (4 / 3) * np.pi * (bin_edges[1:]**3 - bin_edges[:-1]**3)
    normalized_hist = hist / (volumes * density * num_particles)
    return bin_centers, normalized_hist

#### 1. Characterize your model's behavior at N/V = 0.5 in the NVT ensemble.
a. Above what temperature is your system "hot"? How do you know?<br>
b. Below what temperature is your system "frozen"? How do you know?<br>
c. How does the system's total energy, potential energy, kinetic energy, heat capacity, and structure vary from frozen to too hot?<br>

In [ ]:
T_01 = Simulation('T_0.1',T=0.1)
T_01.run(500000)

T_05 = Simulation('T_0.5',T=0.5)
T_05.run(500000)

T_1 = Simulation('T_1.0',T=1.0)
T_1.run(500000)

T_5 = Simulation('T_5.0',T=5.0)
T_5.run(1000000)

T_10 = Simulation('T_10.0',T=10.0)
T_10.run(1000000)

In [ ]:
default_box_length = 25.198421478271484

In [ ]:
# Note: I had to calculate rdfs in separate cells. Running a loop for some reason leads to an error.
# I have to re-run the cell that defines the rdf function between each calculation. I'm not sure what is causing
# this bug, unfortunately, but that is the reasoning for this sub-optimal code structure.

bins,hist = rdf('T_0.1/final_positions.txt',default_box_length,0.5)
rdf = np.array([bins,hist])
np.savetxt("T_0.1/rdf.txt", rdf.T, delimiter=" ")

In [ ]:
df = pd.read_csv('T_0.1/rdf.txt',names=('bins','freq'),sep=None)
plt.plot(df['bins'],df['freq'])
plt.show()

In [ ]:
bins,hist = rdf('T_0.5/final_positions.txt',default_box_length)
rdf = np.array([bins,hist])
np.savetxt("T_0.5/rdf.txt", rdf.T, delimiter=" ")

In [ ]:
bins,hist = rdf('T_1.0/final_positions.txt',default_box_length)
rdf = np.array([bins,hist])
np.savetxt("T_1.0/rdf.txt", rdf.T, delimiter=" ")

In [ ]:
bins,hist = rdf('T_5.0/final_positions.txt',default_box_length)
rdf = np.array([bins,hist])
np.savetxt("T_5.0/rdf.txt", rdf.T, delimiter=" ")

In [ ]:
bins,hist = rdf('T_10.0/final_positions.txt',default_box_length)
rdf = np.array([bins,hist])
np.savetxt("T_10.0/rdf.txt", rdf.T, delimiter=" ")

In [ ]:
file_list = ['T_0.1','T_0.5','T_1.0','T_5.0','T_10.0']


for file in file_list:
    df = pd.read_csv(f'{file}/rdf.txt',names=('bins','freq'),sep=None)
    plt.plot(df['bins'],df['freq'])
    plt.show()
    


#### 2. Characterize finite size effects.
a. How small is too small to be correct? How large is too large to be practical?

#### 3. Contrast your system with an ideal gas.<br>
a. How does the structure of your model vary with state, and how does it compare to particles with no interactions?<br>
b. Does the heat capacity of your system depend on state differently than an ideal gas?<br>
c. Can you derive or numerically determine an equation of state?

#### 4. Summarize your observations, challenges, and any revelations you had while working towards 1-3. Which specific simulation and state point was your favorite and why?